# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує)
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор.

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [1]:
pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [52]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Завантаження змінних з .env
load_dotenv()

# Отримання значень
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# Формування рядка підключення
connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)

# Перевірка
print("✅ Підключення успішне!" if engine is not None else "❌ Помилка підключення.")


✅ Підключення успішне!


In [53]:
from sqlalchemy import create_engine, text
def update_customer_phone(engine, customer_number, new_phone):
    """
    Оновлюю телефон клієнта з використанням параметризованого запиту через text()
    """
    with engine.connect() as conn:
        with conn.begin():
            # Оновлення телефону
            update_query = text("""
                UPDATE customers
                SET phone = :new_phone
                WHERE customerNumber = :cust_num
            """)
            conn.execute(update_query, {
                'new_phone': new_phone,
                'cust_num': customer_number
            })
            print(f"Телефон клієнта #{customer_number} успішно оновлено на {new_phone}")

# Виклик функції — оновлення телефону для клієнта №103
update_customer_phone(engine, 103, "+380631111111")

# SELECT-запит для перевірки
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT customerNumber, customerName, phone
        FROM customers
        WHERE customerNumber = 103
    """))

    for row in result:
        print(row)

Телефон клієнта #103 успішно оновлено на +380631111111
(103, 'Atelier graphique', '+380631111111')


In [54]:
from sqlalchemy import create_engine, text

# Функція оновлення email без логування
def update_customer_email(engine, customer_number, new_email):
    """
    Оновлює email клієнта (якщо поле існує)
    """
    with engine.connect() as conn:
        with conn.begin():
            # Перевірка чи існує поле email у таблиці customers
            check_column_query = text("""
                SELECT COLUMN_NAME 
                FROM INFORMATION_SCHEMA.COLUMNS 
                WHERE TABLE_NAME = 'customers' 
                  AND COLUMN_NAME = 'email' 
                  AND TABLE_SCHEMA = 'classicmodels'
            """)
            column_exists = conn.execute(check_column_query).fetchone()
            if not column_exists:
                print("Поле 'email' не існує в таблиці 'customers'.")
                return

            # Оновлення email
            update_query = text("""
                UPDATE customers
                SET email = :new_email
                WHERE customerNumber = :cust_num
            """)
            conn.execute(update_query, {
                'new_email': new_email,
                'cust_num': customer_number
            })

            print(f"Email клієнта #{customer_number} оновлено на {new_email}")

# Виклик функції
update_customer_email(engine, 103, "new_email@example.com")

Поле 'email' не існує в таблиці 'customers'.


In [55]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT phone FROM customers WHERE customerNumber = :cust_num
    """), {'cust_num': 103})

    row = result.fetchone()
    if row:
        print(f"Телефон клієнта №103: {row[0]}")
    else:
        print("Клієнта з номером 103 не знайдено.")


Телефон клієнта №103: +380631111111


In [56]:
from sqlalchemy import create_engine, text

# Замініть значення на свої, якщо потрібно
connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS customer_phone_log (
            log_id INT AUTO_INCREMENT PRIMARY KEY,
            customerNumber INT NOT NULL,
            old_phone VARCHAR(50),
            new_phone VARCHAR(50),
            changed_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """))
    print("Таблиця customer_phone_log створена або вже існує.")


Таблиця customer_phone_log створена або вже існує.


In [57]:
update_customer_phone_with_log(engine, 103, "+380631112233")


Телефон клієнта #103 успішно оновлено з +380631111111 на +380631112233


In [58]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT customerNumber, phone
        FROM customers
        WHERE customerNumber = 103
    """))
    for row in result:
        print(row)


(103, '+380631112233')


In [59]:
import pandas as pd
from sqlalchemy import text

def customer_phone_history(engine, customer_number):
    print(f"Запит історії для клієнта #{customer_number}")
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT log_id, customerNumber, old_phone, new_phone, changed_at
            FROM customer_phone_log
            WHERE customerNumber = :cust_num
            ORDER BY changed_at DESC
        """), {'cust_num': customer_number})

        rows = result.fetchall()
        print(f"Знайдено записів: {len(rows)}")  # Відладка

        if not rows:
            print(f"Немає історії змін для клієнта #{customer_number}")
            return None

        df = pd.DataFrame(rows, columns=["log_id", "customerNumber", "old_phone", "new_phone", "changed_at"])
        print(df)
        return df

# Виклик
customer_phone_history(engine, 103)


Запит історії для клієнта #103
Знайдено записів: 3
   log_id  customerNumber      old_phone      new_phone          changed_at
0       3             103  +380631111111  +380631112233 2025-08-08 08:41:14
1       2             103  +380631111111  +380631112233 2025-08-07 08:05:12
2       1             103  +380631111111  +380631112233 2025-08-07 07:59:33


,log_id,customerNumber,old_phone,new_phone,changed_at
0,3,103,+380631111111,+380631112233,2025-08-08 08:41:14
1,2,103,+380631111111,+380631112233,2025-08-07 08:05:12
2,1,103,+380631111111,+380631112233,2025-08-07 07:59:33


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [98]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime

# Підключення до БД
connection_str = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)

def create_order_with_items(engine, customer_number, order_items):
    """
    Створює нове замовлення з товарами та повертає DataFrame з деталями.
    """
    with engine.connect() as conn:
        with conn.begin():  # Починаємо транзакцію

            # Генеруємо новий orderNumber
            result = conn.execute(text("SELECT MAX(orderNumber) FROM orders"))
            max_order_number = result.scalar() or 10000
            new_order_number = max_order_number + 1

            # Створюємо запис у таблиці orders
            now = datetime.now().strftime("%Y-%m-%d")
            required_date = now
            conn.execute(text("""
                INSERT INTO orders (orderNumber, orderDate, requiredDate, status, customerNumber)
                VALUES (:orderNumber, :orderDate, :requiredDate, 'In Process', :customerNumber)
            """), {
                'orderNumber': new_order_number,
                'orderDate': now,
                'requiredDate': required_date,
                'customerNumber': customer_number
            })

            # Додаємо позиції замовлення
            inserted_items = []
            for idx, item in enumerate(order_items):
                product_code = item["productCode"]
                quantity = item["quantityOrdered"]
                price = item["priceEach"]

                # Перевірка наявності
                stock_result = conn.execute(text("""
                    SELECT quantityInStock FROM products WHERE productCode = :productCode
                """), {'productCode': product_code}).scalar()

                if stock_result is None:
                    raise ValueError(f"❌ Товар з кодом {product_code} не знайдено")
                if stock_result < quantity:
                    raise ValueError(f"❌ Недостатньо товару {product_code} на складі. Є лише {stock_result}")

                # Додаємо до orderdetails
                conn.execute(text("""
                    INSERT INTO orderdetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                    VALUES (:orderNumber, :productCode, :quantityOrdered, :priceEach, :orderLineNumber)
                """), {
                    'orderNumber': new_order_number,
                    'productCode': product_code,
                    'quantityOrdered': quantity,
                    'priceEach': price,
                    'orderLineNumber': idx + 1
                })

                # Оновлення залишку
                conn.execute(text("""
                    UPDATE products
                    SET quantityInStock = quantityInStock - :quantity
                    WHERE productCode = :productCode
                """), {
                    'quantity': quantity,
                    'productCode': product_code
                })

                # Для DataFrame
                inserted_items.append({
                    'orderNumber': new_order_number,
                    'productCode': product_code,
                    'quantityOrdered': quantity,
                    'priceEach': price,
                    'orderLineNumber': idx + 1
                })

            print(f"Замовлення #{new_order_number} успішно створено з {len(order_items)} товар(ами)")

        # 4. Повертаємо DataFrame
        df = pd.DataFrame(inserted_items)
        print("\nДеталі замовлення:")
        print(df)
        return df


In [99]:
order_items = [
    {"productCode": "S10_1678", "quantityOrdered": 5, "priceEach": 95.70},
    {"productCode": "S18_2248", "quantityOrdered": 3, "priceEach": 55.09}
]

create_order_with_items(engine, customer_number=103, order_items=order_items)


Замовлення #10438 успішно створено з 2 товар(ами)

Деталі замовлення:
   orderNumber productCode  quantityOrdered  priceEach  orderLineNumber
0        10438    S10_1678                5      95.70                1
1        10438    S18_2248                3      55.09                2


,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10438,S10_1678,5,95.70,1
1,10438,S18_2248,3,55.09,2


In [101]:
order_number = 10436  # останній номер

with engine.connect() as conn:
    print("Замовлення:")
    df_order = pd.read_sql(f"SELECT * FROM orders WHERE orderNumber = {order_number}", conn)
    print(df_order)

    print("\nДеталі замовлення:")
    df_details = pd.read_sql(f"SELECT * FROM orderdetails WHERE orderNumber = {order_number}", conn)
    print(df_details)

    print("\nЗалишки товарів після зменшення:")
    df_stock = pd.read_sql("""
        SELECT productCode, quantityInStock FROM products
        WHERE productCode IN ('S10_1678', 'S18_2248')
    """, conn)
    print(df_stock)


Замовлення:
   orderNumber   orderDate requiredDate shippedDate      status comments  \
0        10436  2025-08-08   2025-08-08        None  In Process     None   

   customerNumber  
0             103  

Деталі замовлення:
   orderNumber productCode  quantityOrdered  priceEach  orderLineNumber
0        10436    S10_1678                5      95.70                1
1        10436    S18_2248                3      55.09                2

Залишки товарів після зменшення:
  productCode  quantityInStock
0    S10_1678             7903
1    S18_2248              522
